# RouteE API Walkthrough

This notebook demonstrates how to use the RouteE API. 

### Requirements: 
- [Python](https://www.python.org/): 3.8.X
- [requests](https://docs.python-requests.org/en/latest/): 2.25.X
- [pandas](https://pandas.pydata.org/): 1.2.X
- A developer.nrel.gov API key. Sign-up [here](https://developer.nrel.gov/signup/).

For more detailed information see the [API documentation](https://developer.nrel.gov/docs/transportation/routee-v1/) or the [API key rate limits page](https://developer.nrel.gov/docs/rate-limits/).

## Design and Intend Usage Philosophy

This tool was designed with a specific application in mind. The overall workflow is:

1. Take the user's tabular route or transport network data, convert it to JSON format
2. POST to the API the route/network data with additional JSON parameters that define RouteE model application.
3. Return the API response in JSON format with road segment energy estimates and metadata. Then, convert the response JSON back to tabular format.
4. Join (by attribute) with your original data to append the energy cost information.

## A. route endpoint example:

In [1]:
# Setup
import requests
import warnings
import json

import pandas as pd
from pprint import pprint
from pathlib import Path

warnings.filterwarnings('ignore')

your_api_key = 'DEMO_KEY'

DEV_URL_ROUTE = f'https://developer.nrel.gov/api/routee/v2/route?api_key={your_api_key}'

## Design and Intend Usage Philosophy

This tool was designed with a specific application in mind. The overall workflow is:

1. Take the user's tabular route or transport network data, convert it to JSON format
2. POST to the API the route/network data with additional JSON parameters that define RouteE model application.
3. Return the API response in JSON format with road segment energy estimates and metadata. Then, convert the response JSON back to tabular format.
4. Join (by attribute) with your original data to append the energy cost information.

In [5]:
raw_network = pd.read_csv(r'..\test_data\test_route.csv') #pretending this represents a segment-level road network

FT_TO_MILES = 0.000189394
raw_network['volume'] = 50 #assuming constant traffic on all road segments
raw_network['grade'] = 0.0 #assuming flat road grade

ids = list(raw_network['net_id'])
speeds = list(raw_network['speed_mph'])
miles = list(raw_network['length_mi'])
grades = list(raw_network['grade_new'])
volumes = list(raw_network['volume'])

raw_network.head(3)

,sampno,tripno,net_id,length_mi,speed_mph,miles_new,gpsspeed_new,volume_new,grade_new
0,11394,1,23268,0.011648,25,0.011648,25,50,0.0
1,11394,1,169248,0.056629,50,0.056629,50,50,0.0
2,11394,1,212054,0.004110,45,0.004110,45,50,0.0


In [6]:
request_route_json = request_route.to_dict(orient='list')

print("Now, each key in this object represnts the column name, and the corresponding value is an ordered array of the tabular data values...")
print(request_route_json.keys())

print("\nnet_id values: ")
print(request_route_json['net_id'])

Now, each key in this object represnts the column name, and the corresponding value is an ordered array of the tabular data values...
dict_keys(['net_id', 'miles_new', 'gpsspeed_new', 'grade_new'])

net_id values: 
[23268, 169248, 212054, 78644, 220927, 105336, 169177, 241740, 104671, 204282, 76309, 216556, 216556, 104411, 267287, 294133, 73830, 3483, 95298, 121242, 145347, 285510, 100578, 169693, 13838, 53720, 53720, 56767, 243066, 228406, 86346, 311372, 96972, 8037, 257441, 284938, 61371, 179737, 179972, 179972, 67737, 171114, 136953, 79547, 10885, 102705, 67903, 168516, 70025, 62517, 199702, 284547, 202258, 196206, 67816, 65129, 310230, 71209, 276847, 44390, 129580, 299035, 196825, 290084, 119061]


## 2. POST to the API the route/network data with additional JSON parameters that define RouteE model application.

Data is POSTed to the `/route` endpoint in JSON format. Each request should contain the following JSON keys: 

1. "route": the JSON route data from the above.
2. "model": the string representation of the desired vehicle model. The options are:
    a. "cv": conventional gasoline light-duty vehicle
    b. "diesel": diesel light-duty vehicle
    c. "bev": battery electric light-duty vehicle
    d. "hev": hybrid light-duty vehicle
    e. "phev": plug-in hybrid light-duty vehicle
    
3. "field_mapper": this is where you tell the API which of your fields match with the specific field names the routee Python code looks for. In this example:
    a. the input data calls the MPH speed field "gpsspeed_new". RouteE looks for "gpsspeed", so the value in the `field_mapper` object would written `"gpsspeed": "gpsspeed_new"`.
    b. repeat for `"miles": "miles_new", "grade": "grade_new", "id": "net_d"`.
4. "omit_grade" (optional Boolean): This can be used and set to True if you have no grade information on your road segments. By default, it is False and assumes you will be passing in valid `"grade"` data.

In [7]:
# Construct the application/json content-type header object

request_route_data = {
    "route": request_route_json,
    "model": "diesel", #starting with diesel for this example
    "field_mapper": {
        "miles": "miles_new",
        "gpsspeed": "gpsspeed_new",
        "grade":"grade_new",
        "id": "net_id"
    }
}

# POST to the API
headers = {'content-type': 'application/json'}
p = requests.post(DEV_URL_ROUTE,  json=request_route_data, headers=headers)
print(p.status_code) #Hope for 200

200


## 3. Return the API response in JSON format with road segment energy estimates and metadata. Convert the response JSON back to tabular format.

In [8]:
if p.status_code == 200:
    print('Metadata: \n')
    pprint(p.json()['output_metadata'])

    print('\nData: ')
    new_df = pd.DataFrame(p.json()['route'])
    print(new_df.head())
else:
    print(p.text)

Metadata: 

{'Data Dictionary': {'Output fields': {'energy_estimate': 'Segment level '
                                                          'energy estimate'}},
 'Energy Unit': 'gallons',
 'RouteE version': '0.4.0',
 'Total Energy': '0.1009',
 'Vehicle description': '2016_BMW_328d_4cyl_2WD'}

Data: 
   net_id  energy_estimate
0   23268         0.000410
1  169248         0.001416
2  212054         0.000103
3   78644         0.001816
4  220927         0.000112


## 4. Join (by attribute) with your original data to append the energy cost information.

In [9]:
# Inner join the resulting data with the input data on the "net_id" column
joined_data = request_route.merge(new_df, on='net_id', how='inner') 

joined_data.head()

,net_id,miles_new,gpsspeed_new,grade_new,energy_estimate
0,23268,0.011648,25,0.0,0.000410
1,169248,0.056629,50,0.0,0.001416
2,212054,0.004110,45,0.0,0.000103
3,78644,0.072633,50,0.0,0.001816
4,220927,0.004489,45,0.0,0.000112


## B. network endpoint example:

This follows the same design as the /route endpoint, with a few changes...

In [15]:
DEV_URL_ROUTE = f'https://developer.nrel.gov/api/routee/v1/network?api_key={your_api_key}'

In [ ]:
# Converting the test data .csv to a pandas data table.
test_route_path = Path('../../test_data/test_route/test_route.csv')
raw_network = pd.read_csv(test_route_path) #test data that could represent a network or a route. Provided separately

FT_TO_MILES = 0.000189394

# Current RouteE models require segment-level distance (in miles), speed (in MPH), and grade (%) to return an energy estimate
raw_network['miles_new'] = raw_network.length_ft * FT_TO_MILES 
raw_network['gpsspeed_new'] = raw_network.speed_mph #can just be posted speed
raw_network['grade_new'] = 0.0 #assuming flat grade
raw_network['volume'] = 25 #assuming constant volume on each road segment

request_network = raw_network[['net_id', 'miles_new', 'gpsspeed_new', 'grade_new']] #trimming to only necessary columns

print("What your data might look like in your database: ")
request_route.head(3)